In [1]:
import numpy as np
from datetime import datetime

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [2]:
# LogicGate Class

class LogicGate:
    
    # 생성자
    # xdata, tdata => numpy.array(...)
    def __init__(self, gate_name, xdata, tdata, input_nodes, hidden_1_nodes, output_nodes, learning_rate, iteration_count):
        
        self.name = gate_name
        
        # 입력 데이터, 정답 데이터 초기화
        self.xdata = xdata.reshape(4, 2)
        self.tdata = tdata.reshape(4, 1)
        
        # 2층 hidden layer unit : 
        # 가중치 W, 바이어스 b 초기화
        self.W2 = np.random.rand(input_nodes, hidden_1_nodes)  
        self.b2 = np.random.rand(hidden_1_nodes)
        
        # 3층 output layer unit : 1 개 
        self.W3 = np.random.rand(hidden_1_nodes, output_nodes)
        self.b3 = np.random.rand(output_nodes)
                        
        # 학습률 learning rate 초기화
        self.learning_rate = learning_rate
        
        # 반복횟수 초기화
        self.iteration_count = iteration_count
    
        print(self.name + " object is created")
        
    # 손실함수
    def feed_forward(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )
    
    # obtain W and b
    def get_W_b(self):
        
        return self.W2,  self.b2, self.W3, self.b3
    
    # 손실 값 계산
    def loss_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )
    
    # query, 즉 미래 값 예측 함수
    def predict(self, xdata):
        
        z2 = np.dot(xdata, self.W2) + self.b2
        a2 = sigmoid(z2)
        
        z3 = np.dot(a2, self.W3) + self.b3
        y = a3 = sigmoid(z3)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

    # accuracy method 1
    def accuracy1(self, test_xdata, test_tdata):
        
        matched_list = []
        not_matched_list = []
        
        for index in range(len(test_xdata)):
            
            (real_val, logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_result = len(matched_list) / len(test_xdata)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list
    
    # accuracy method 2
    def accuracy2(self, test_data):
        
        matched_list = []
        not_matched_list = []
        
        input_data = test_data[ :, 0:-1]
        target_data = test_data[ :, -1]
        
        for index in range(len(input_data)):
            
            (real_val, logical_val) = self.predict(input_data[index])
            
            if logical_val == target_data[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_result = len(matched_list) / len(input_data)
        
        print("Accuracy => ", accuracy_result)
        
        return matched_list, not_matched_list
        
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self):
        
        f = lambda x : self.feed_forward()
        
        print("Initial loss value = ", self.loss_val())
        
        start_time = datetime.now()
        
        for step in  range(self.iteration_count):
            
            self.W2 -= self.learning_rate * numerical_derivative(f, self.W2)
        
            self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        
            self.W3 -= self.learning_rate * numerical_derivative(f, self.W3)
    
            self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)
    
            if (step % 500 == 0):
                print("step = ", step, "loss value = ", self.loss_val())
                
        end_time = datetime.now()
        
        print("")
        print("Elapsed Time => ", end_time - start_time)

In [3]:
and_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
and_tdata = np.array([0, 0, 0, 1])

input_nodes = 2
hidden_1_nodes = 8
output_nodes = 1

lr = 1e-1  # learning rate
count = 10001  # iteration count

# AND Gate 객체 생성 및 training
and_obj = LogicGate("AND", and_xdata, and_tdata, input_nodes, hidden_1_nodes, output_nodes, lr, count)

and_obj.train()

AND object is created
Initial loss value =  11.69008096769096
step =  0 loss value =  7.905237398723822
step =  500 loss value =  0.163099007050745
step =  1000 loss value =  0.04337260646364584
step =  1500 loss value =  0.022793663090724524
step =  2000 loss value =  0.0149959136527623
step =  2500 loss value =  0.01101169716154074
step =  3000 loss value =  0.008627465246829755
step =  3500 loss value =  0.007053840508837074
step =  4000 loss value =  0.005943574679065932
step =  4500 loss value =  0.005121419480603846
step =  5000 loss value =  0.004489881631401587
step =  5500 loss value =  0.003990623992516337
step =  6000 loss value =  0.0035867046120384135
step =  6500 loss value =  0.0032536448646948686
step =  7000 loss value =  0.0029746110158193504
step =  7500 loss value =  0.002737662452568424
step =  8000 loss value =  0.0025341042024681106
step =  8500 loss value =  0.0023574605504365336
step =  9000 loss value =  0.0022028134799142894
step =  9500 loss value =  0.00206

In [4]:
# AND Gate prediction
print(and_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 0], [1, 0, 0], [1, 1, 1] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 0, 0, 1])

(true_list, false_list) = and_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = and_obj.accuracy2(test_data)

AND 

Accuracy =>  1.0
Accuracy =>  1.0


In [5]:
or_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
or_tdata = np.array([0, 1, 1, 1])

input_nodes = 2
hidden_1_nodes = 8
output_nodes = 1

lr = 1e-1  # learning rate
count = 10001  # iteration count

# OR Gate 객체 생성 및 training
or_obj = LogicGate("OR", or_xdata, or_tdata, input_nodes, hidden_1_nodes, output_nodes, lr, count)

or_obj.train()

OR object is created
Initial loss value =  3.4399552130171096
step =  0 loss value =  3.095318381014899
step =  500 loss value =  0.13839149385410668
step =  1000 loss value =  0.042382165434642335
step =  1500 loss value =  0.023411386288769834
step =  2000 loss value =  0.015790283328216036
step =  2500 loss value =  0.011765659051113545
step =  3000 loss value =  0.009304804021568523
step =  3500 loss value =  0.0076556671766316265
step =  4000 loss value =  0.006478854822481096
step =  4500 loss value =  0.0055997792406362636
step =  5000 loss value =  0.004919850833001289
step =  5500 loss value =  0.004379358200589502
step =  6000 loss value =  0.003940108008257268
step =  6500 loss value =  0.0035765772434726835
step =  7000 loss value =  0.0032710861792915264
step =  7500 loss value =  0.0030110150166789383
step =  8000 loss value =  0.0027871234960588206
step =  8500 loss value =  0.0025924962943964956
step =  9000 loss value =  0.002421858834274174
step =  9500 loss value =  

In [6]:
# OR Gate prediction
print(or_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 1] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 1])

(true_list, false_list) = or_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = or_obj.accuracy2(test_data)

OR 

Accuracy =>  1.0
Accuracy =>  1.0


In [7]:
nand_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
nand_tdata = np.array([1, 1, 1, 0])

input_nodes = 2
hidden_1_nodes = 8
output_nodes = 1

lr = 1e-1  # learning rate
count = 10001  # iteration count

# NAND Gate 객체 생성 및 training
nand_obj = LogicGate("NAND", nand_xdata, nand_tdata, input_nodes, hidden_1_nodes, output_nodes, lr, count)

nand_obj.train()

NAND object is created
Initial loss value =  3.852812032196662
step =  0 loss value =  3.3695674150981265
step =  500 loss value =  0.17253108311745635
step =  1000 loss value =  0.049024053563910694
step =  1500 loss value =  0.026296933217919996
step =  2000 loss value =  0.017476696547669777
step =  2500 loss value =  0.012911858035288291
step =  3000 loss value =  0.010157601298117127
step =  3500 loss value =  0.008329165980266767
step =  4000 loss value =  0.007033492409438232
step =  4500 loss value =  0.006070777262903445
step =  5000 loss value =  0.0053292457040264715
step =  5500 loss value =  0.004741715055661299
step =  6000 loss value =  0.004265484653661866
step =  6500 loss value =  0.0038721719397812673
step =  7000 loss value =  0.003542206478767972
step =  7500 loss value =  0.003261674365866504
step =  8000 loss value =  0.003020423176456373
step =  8500 loss value =  0.002810878174949804
step =  9000 loss value =  0.0026272774845950545
step =  9500 loss value =  0.

In [8]:
# NAND Gate prediction
print(nand_obj.name, "\n")

test_data = np.array([ [0, 0, 1], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 1, 1, 1, 0])

(true_list, false_list) = nand_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = nand_obj.accuracy2(test_data)

NAND 

Accuracy =>  1.0
Accuracy =>  1.0


In [4]:
xor_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
xor_tdata = np.array([0, 1, 1, 0])

input_nodes = 2
hidden_1_nodes = 8
output_nodes = 1

lr = 1e-1  # learning rate
count = 10001  # iteration count

#learning rate 와 epoch 그리고 hidden node수를 조절해야한다
# learning rate 를 조절함으로써 local minimum 을 뛰어넘는걸 만들어내야한다
# XOR Gate 객체 생성 및 training
xor_obj = LogicGate("XOR", xor_xdata, xor_tdata, input_nodes, hidden_1_nodes, output_nodes, lr, count)

xor_obj.train()

XOR object is created
Initial loss value =  6.622081048124219
step =  0 loss value =  4.911641983912656
step =  500 loss value =  2.3675988795926584
step =  1000 loss value =  0.3737507450790143
step =  1500 loss value =  0.12836788860453235
step =  2000 loss value =  0.07326772451181199
step =  2500 loss value =  0.050388611528105795
step =  3000 loss value =  0.03809200438334799
step =  3500 loss value =  0.030481494334467375
step =  4000 loss value =  0.02533288171221805
step =  4500 loss value =  0.021629786242871797
step =  5000 loss value =  0.018844557664573408
step =  5500 loss value =  0.01667706223626949
step =  6000 loss value =  0.01494442147067266
step =  6500 loss value =  0.013529051002520193
step =  7000 loss value =  0.012352013086055717
step =  7500 loss value =  0.011358402028630149
step =  8000 loss value =  0.010508878656775392
step =  8500 loss value =  0.009774538768119168
step =  9000 loss value =  0.009133680884080059
step =  9500 loss value =  0.00856970233227

In [10]:
# XOR Gate prediction
print(xor_obj.name, "\n")

test_data = np.array([ [0, 0, 0], [0, 1, 1], [1, 0, 1], [1, 1, 0] ])

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 0])

(true_list, false_list) = xor_obj.accuracy1(test_xdata, test_tdata)
(true_list, false_list) = xor_obj.accuracy2(test_data)

XOR 

Accuracy =>  1.0
Accuracy =>  1.0
